In [40]:
import pandas as pd
from os.path import join

from sklearn import svm
from pandas import DataFrame
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
import os
os.chdir('/home/potusvn/Projects/kickstarter_prediction')

In [ ]:
PROJECT_ROOT = '/home/potusvn/Projects/kickstarter_prediction'
DATA_RAW_ROOT = os.path.join(PROJECT_ROOT, 'data', 'raw')
DATA_PREPROCESSED_ROOT = os.path.join(PROJECT_ROOT, 'data', 'pre_processed')
DATA_ENGINEER_ROOT = os.path.join(PROJECT_ROOT, 'data', 'engineered')
DATA_EXTERNAL_ROOT = os.path.join(PROJECT_ROOT, 'data', 'external')
DATA_SPLIT_ROOT = os.path.join(PROJECT_ROOT, 'data', 'data_spliting')

MODELS_ROOT = os.path.join(PROJECT_ROOT, 'models')

#export PYTHONPATH=$PYTHONPATH:'pwd'

train_set = join(DATA_SPLIT_ROOT, 'train.csv')
test_set = join(DATA_SPLIT_ROOT, 'test.csv')

train = pd.read_csv(train_set, encoding='latin1', low_memory=True)
test = pd.read_csv(test_set, encoding='latin1', low_memory=True)

features = train.drop(['success'], axis=1)
targets = train['success']

In [8]:
targets.value_counts() / len(targets)

0    0.595288
1    0.404712
Name: success, dtype: float64

In [10]:
test.success.value_counts() / test.shape[0]

0    0.598058
1    0.401942
Name: success, dtype: float64

In [3]:
model = svm.SVC(gamma='scale')
scores = cross_val_score(model, features, targets, cv=5)

NameError: name 'args' is not defined

In [11]:
print('SVM 5-fold scores: ', scores)
print('SVM accuracy: %.4f%% (+/- %.4f%%)' % (scores.mean() * 100.0, scores.std() * 2 * 100.0))

SVM 5-fold scores:  [0.75096371 0.74743189 0.75121678 0.74466436 0.74703336]
SVM accuracy: 74.8262% (+/- 0.4994%)


## Check the with SVM Quantile

In [12]:
train_quantile = pd.read_csv('data/data_quantile_transform/train_quantile_transform.csv', encoding='latin1', low_memory=True)
test_quantile = pd.read_csv('data/data_quantile_transform/test_quantile_transform.csv', encoding='latin1', low_memory=True)
val_quantile = pd.read_csv('data/data_quantile_transform/val_quantile_transform.csv', encoding='latin1', low_memory=True)

In [15]:
train_quantile_large = pd.concat([train_quantile,val_quantile], axis=0)
train_quantile_large.head(10)

,usd_goal_real,success,contain_special_symbols,name_length,num_vowels,num_cap,num_whitespace,contain_bad_word,subjectivity,polarity,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,0.508364,0,0,0.236737,0.381882,0.139640,9.109109e-02,0,1.000000e-07,0.437437,...,0,0,0,0,0,0,0,0,0,1
1,0.999039,0,0,0.418418,0.499499,0.565065,3.278278e-01,0,1.000000e-07,0.437437,...,0,0,0,0,0,0,0,0,0,1
2,0.473473,0,0,0.278779,0.499499,0.435435,4.504505e-01,0,8.623624e-01,0.837337,...,0,0,0,0,0,0,0,0,0,1
3,0.823582,1,1,0.881381,0.961962,0.765265,7.722723e-01,0,7.887888e-01,0.020521,...,0,0,0,0,0,0,0,0,0,1
4,0.008008,1,0,0.278779,0.189189,0.286286,2.007007e-01,0,1.000000e-07,0.437437,...,0,0,0,0,0,0,0,0,0,1
5,0.192212,0,1,0.922422,0.855355,0.765265,7.722723e-01,1,9.999999e-01,1.000000,...,0,0,0,0,0,0,0,0,0,1
6,0.658158,0,0,0.023524,0.075075,0.832332,1.000000e-07,0,1.000000e-07,0.437437,...,0,0,0,0,0,0,0,0,0,1
7,0.618619,0,1,0.828328,0.961962,0.435435,9.289289e-01,0,1.000000e-07,0.437437,...,0,0,0,0,0,0,0,0,0,1
8,0.219192,0,0,0.061562,0.035536,0.286286,9.109109e-02,0,1.000000e-07,0.437437,...,0,0,0,0,0,0,0,0,0,0
9,0.900400,0,1,0.811812,0.758759,0.765265,8.613614e-01,0,1.000000e-07,0.437437,...,0,0,0,0,0,0,0,0,0,1


In [21]:
train_x = train_quantile_large.drop(['success'], axis=1)
train_y = train_quantile_large.success
test_x = test_quantile.drop(['success'], axis=1)
test_y = test_quantile.success

In [37]:
train_x.shape

(232170, 221)

In [41]:
# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
SVM = svm.SVC()
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('svm', SVM)])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [int(i*221/100) for i in range(20, 100, 20)],
    'SVM__kernel' : ['rbf', 'poly', 'linear'],
    'SVM__C' : [0.001, 0.01, 0.1, 1, 10],
    'SVM__gamma': [0.001, 0.01, 0.1, 1]
    
}
search = GridSearchCV(pipe, param_grid, iid=False, cv=5,
                      return_train_score=False, n_jobs=-1)
search.fit(train_x, train_y)

ValueError: Invalid parameter SVM for estimator Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

/home/potusvn/.conda/envs/kickstarter/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [27]:
search

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'pca__n_components': [5, 20, 30, 40, 50, 64]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

In [ ]:
# Plot the PCA spectrum
pca.fit(X_digits)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(pca.explained_variance_ratio_, linewidth=2)
ax0.set_ylabel('PCA explained variance')

ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,